Feature Engineering

Feature Scaling

Model Development

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import boxcox
from scipy.stats import yeojohnson
from spici import dtex
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)

In [89]:
dataset_tr = pd.read_csv('train.csv')
dataset_te = pd.read_csv('test.csv')
x_tr = dataset_tr.copy()
x_te = dataset_te.copy()

In [90]:
x_tr.drop(['Id', 'SalePrice'], axis = 1, inplace = True)
x_te.drop('Id', axis = 1, inplace = True)

In [91]:
dict = dtex.feature_distribution(x_tr, 25, ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold'])
cont = dict['Continous Features']
disc_o = dict['Discrete Object Features']
disc_n = dict['Discrete Numerical Features']
disc = dict['Discrete Features']
temp = dict['Temporal Features']
print('\n\n')
dict = dtex.feature_distribution(x_te, 25, ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold'])
cont_te = dict['Continous Features']
disc_o_te = dict['Discrete Object Features']
disc_n_te = dict['Discrete Numerical Features']
disc_te = dict['Discrete Features']
temp_te = dict['Temporal Features']

Object Features: ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
Numerical Feature: ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 

In [92]:
l = {}
for feature in cont:
    print(feature)
    print('\t', 'has skewness', x_tr[feature].skew())
    sp = np.abs(x_tr[feature].skew())
    la = np.sqrt(x_tr[feature])
    sa = np.abs(la.skew())
    if sa>sp:
        l[feature] = 'sa = '+str(sa)+' > sp = '+str(sp)
        print(feature, ': sa = '+str(sa)+' > sp = '+str(sp))
    elif sa == sp:
        l[feature] = 'sa = sp, '+str(sp)
        print(feature, ': sa = sp, '+str(sp))
    else:
        x_tr[feature] = np.sqrt(x_tr[feature])
        print('\t', x_tr[feature].skew(), 'skewness after sqrt transformation')
print(l)

LotFrontage
	 has skewness 2.163569142324884
	 0.37938772516667946 skewness after sqrt transformation
LotArea
	 has skewness 12.207687851233496
	 4.139713548289236 skewness after sqrt transformation
MasVnrArea
	 has skewness 2.669084210182863
	 1.0402388666895226 skewness after sqrt transformation
BsmtFinSF1
	 has skewness 1.685503071910789
	 -0.02994605300118736 skewness after sqrt transformation
BsmtFinSF2
	 has skewness 4.255261108933303
	 3.0368100092936388 skewness after sqrt transformation
BsmtUnfSF
	 has skewness 0.9202684528039037
	 -0.24650610641958765 skewness after sqrt transformation
TotalBsmtSF
	 has skewness 1.5242545490627664
	 -1.2745373951324759 skewness after sqrt transformation
1stFlrSF
	 has skewness 1.3767566220336365
	 0.6242627120029303 skewness after sqrt transformation
2ndFlrSF
	 has skewness 0.8130298163023265
	 0.42233238629172287 skewness after sqrt transformation
GrLivArea
	 has skewness 1.3665603560164552
	 0.5942546260858922 skewness after sqrt transforma

In [93]:
l = {}
for feature in cont_te:
    print(feature)
    print('\t', 'has skewness', x_te[feature].skew())
    sp = np.abs(x_te[feature].skew())
    la = np.sqrt(x_te[feature])
    sa = np.abs(la.skew())
    if sa>sp:
        l[feature] = 'sa = '+str(sa)+' > sp = '+str(sp)
        print(feature, ': sa = '+str(sa)+' > sp = '+str(sp))
    elif sa == sp:
        l[feature] = 'sa = sp, '+str(sp)
        print(feature, ': sa = sp, '+str(sp))
    else:
        x_te[feature] = np.sqrt(x_te[feature])
        print('\t', x_te[feature].skew(), 'skewness after sqrt transformation')
print(l)

LotFrontage
	 has skewness 0.661921074378434
	 -0.19610259072689962 skewness after sqrt transformation
LotArea
	 has skewness 3.115216613500925
	 0.7361217790665913 skewness after sqrt transformation
MasVnrArea
	 has skewness 2.533376725976824
	 1.0702767653071192 skewness after sqrt transformation
BsmtFinSF1
	 has skewness 1.16567669422342
	 0.005843408257318 skewness after sqrt transformation
BsmtFinSF2
	 has skewness 4.04134463064927
	 2.929589726401015 skewness after sqrt transformation
BsmtUnfSF
	 has skewness 0.919916339164844
	 -0.2296680200693724 skewness after sqrt transformation
TotalBsmtSF
	 has skewness 0.8135891485362542
TotalBsmtSF : sa = 1.3358470482790474 > sp = 0.8135891485362542
1stFlrSF
	 has skewness 1.558194572983426
	 0.6481053893187231 skewness after sqrt transformation
2ndFlrSF
	 has skewness 0.9128826344911904
	 0.46951453566634027 skewness after sqrt transformation
GrLivArea
	 has skewness 1.1304024140503506
	 0.5378275212599957 skewness after sqrt transformat

In [94]:
x_tr = dtex.fillmt(x_tr, disc_o, True)
x_te = dtex.fillmt(x_te, disc_o_te, True)
x_tr = dtex.fillmt(x_tr, cont+temp+disc_n, False)
x_te = dtex.fillmt(x_te, cont_te+temp_te+disc_n_te, False)

0.01407s...
0.01693s...
0.01202s...
0.00901s...


In [95]:
dtex.missing_percentage(x_tr)
dtex.missing_percentage(x_te)

In [96]:
x_tr = dtex.rare_categorical(x_tr, disc_o)
x_te = dtex.rare_categorical(x_te, disc_o_te)

Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "rare-var"
Rare features are marked "ra

In [97]:
x_tr = dtex.category_processing(x_tr, disc_o)
x_te = dtex.category_processing(x_te, disc_o_te)

MSZoning
	RL: 0
	RM: 1
	rare-var: 2
	FV: 3
	RH: 4
Street
	Pave: 0
	rare-var: 1
Alley
	missing: 0
	Grvl: 1
	Pave: 2
LotShape
	Reg: 0
	IR1: 1
	IR2: 2
	rare-var: 3
LandContour
	Lvl: 0
	Bnk: 1
	Low: 2
	HLS: 3
Utilities
	AllPub: 0
	rare-var: 1
LotConfig
	Inside: 0
	FR2: 1
	Corner: 2
	CulDSac: 3
	rare-var: 4
LandSlope
	Gtl: 0
	Mod: 1
	rare-var: 2
Neighborhood
	CollgCr: 0
	rare-var: 1
	Crawfor: 2
	NoRidge: 3
	Mitchel: 4
	Somerst: 5
	NWAmes: 6
	OldTown: 7
	BrkSide: 8
	Sawyer: 9
	NridgHt: 10
	NAmes: 11
	SawyerW: 12
	IDOTRR: 13
	MeadowV: 14
	Edwards: 15
	Timber: 16
	Gilbert: 17
	StoneBr: 18
	ClearCr: 19
	Blmngtn: 20
	BrDale: 21
	SWISU: 22
Condition1
	Norm: 0
	Feedr: 1
	PosN: 2
	Artery: 3
	rare-var: 4
	RRAn: 5
Condition2
	Norm: 0
	rare-var: 1
BldgType
	1Fam: 0
	2fmCon: 1
	Duplex: 2
	TwnhsE: 3
	Twnhs: 4
HouseStyle
	2Story: 0
	1Story: 1
	1.5Fin: 2
	rare-var: 3
	SFoyer: 4
	SLvl: 5
RoofStyle
	Gable: 0
	Hip: 1
	rare-var: 2
RoofMatl
	CompShg: 0
	rare-var: 1
Exterior1st
	VinylSd: 0
	MetalSd: 1
	Wd Sdng:

In [98]:
x_tr.head(10)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60.0,0,8.062258,91.923882,0,0,0,0,0,0,0,0,0,0,0,0,7.0,5.0,2003.0,2003.0,0,0,0,0,0,14.000000,0,0,0,0,0,0,0,26.570661,0,0.000000,12.247449,29.257478,0,0,0,0,29.257478,29.223278,0.0,41.352146,1.0,0.0,2.0,1.0,3.0,1.0,0,8.0,0,0.0,0,0,2003.0,0,2.0,548.0,0,0,0,0.000000,7.810250,0.000000,0.0,0.0,0.0,0,0,0,0.0,2.0,2008.0,0,0
1,20.0,0,8.944272,97.979590,0,0,0,0,0,1,0,1,1,0,0,1,6.0,8.0,1976.0,1976.0,0,0,1,1,1,0.000000,1,0,1,0,0,1,1,31.272992,0,0.000000,16.852300,35.524639,0,0,0,0,35.524639,0.000000,0.0,35.524639,0.0,1.0,2.0,0.0,3.0,1.0,1,6.0,0,1.0,1,0,1976.0,0,2.0,460.0,0,0,0,17.262677,0.000000,0.000000,0.0,0.0,0.0,0,0,0,0.0,5.0,2007.0,0,0
2,60.0,0,8.246211,106.066017,0,0,1,0,0,0,0,0,0,0,0,0,7.0,5.0,2001.0,2002.0,0,0,0,0,0,12.727922,0,0,0,0,0,2,0,22.045408,0,0.000000,20.832667,30.331502,0,0,0,0,30.331502,29.427878,0.0,42.261093,1.0,0.0,2.0,1.0,3.0,1.0,0,6.0,0,1.0,1,0,2001.0,0,2.0,608.0,0,0,0,0.000000,6.480741,0.000000,0.0,0.0,0.0,0,0,0,0.0,9.0,2008.0,0,0
3,70.0,0,7.745967,97.724101,0,0,1,0,0,2,0,2,0,0,0,0,7.0,5.0,1915.0,1970.0,0,0,2,2,1,0.000000,1,0,2,1,1,0,1,14.696938,0,0.000000,23.237900,27.495454,0,1,0,0,31.000000,27.495454,0.0,41.436699,1.0,0.0,1.0,0.0,3.0,1.0,0,7.0,0,1.0,2,1,1998.0,1,3.0,642.0,0,0,0,0.000000,5.916080,16.492423,0.0,0.0,0.0,0,0,0,0.0,2.0,2006.0,0,1
4,60.0,0,9.165151,119.415242,0,0,1,0,0,1,0,3,0,0,0,0,8.0,5.0,2000.0,2000.0,0,0,0,0,0,18.708287,0,0,0,0,0,3,0,25.592968,0,0.000000,22.135944,33.837849,0,0,0,0,33.837849,32.449961,0.0,46.882833,1.0,0.0,2.0,1.0,4.0,1.0,0,9.0,0,1.0,1,0,2000.0,0,3.0,836.0,0,0,0,13.856406,9.165151,0.000000,0.0,0.0,0.0,0,0,0,0.0,12.0,2008.0,0,0
5,50.0,0,9.219544,118.806565,0,0,1,0,0,0,0,4,0,0,0,2,5.0,5.0,1993.0,1995.0,0,0,0,0,1,0.000000,1,0,3,0,0,0,0,27.055499,0,0.000000,8.000000,28.213472,0,0,0,0,28.213472,23.790755,0.0,36.905284,1.0,0.0,1.0,1.0,1.0,1.0,1,5.0,0,0.0,0,0,1993.0,1,2.0,480.0,0,0,0,6.324555,5.477226,0.000000,320.0,0.0,0.0,0,1,1,700.0,10.0,2009.0,0,0
6,20.0,0,8.660254,100.419122,0,0,0,0,0,0,0,5,0,0,0,1,8.0,5.0,2004.0,2005.0,0,0,0,0,2,13.638182,0,0,0,2,0,3,0,37.000000,0,0.000000,17.804494,41.060930,0,0,0,0,41.158231,0.000000,0.0,41.158231,1.0,0.0,2.0,0.0,3.0,1.0,0,7.0,0,1.0,2,0,2004.0,0,2.0,636.0,0,0,0,15.968719,7.549834,0.000000,0.0,0.0,0.0,0,0,0,0.0,8.0,2007.0,0,0
7,60.0,0,8.306624,101.892100,0,0,1,0,0,2,0,6,2,0,0,0,7.0,6.0,1973.0,1973.0,0,0,3,3,2,15.491933,1,0,1,0,0,2,1,29.308702,1,5.656854,14.696938,33.271610,0,0,0,0,33.271610,31.352831,0.0,45.716518,1.0,0.0,2.0,1.0,3.0,1.0,1,7.0,0,2.0,1,0,1973.0,0,2.0,484.0,0,0,0,15.329710,14.282857,15.099669,0.0,0.0,0.0,0,0,1,350.0,11.0,2009.0,0,0
8,50.0,1,7.141428,78.230429,0,0,0,0,0,0,0,7,3,0,0,2,7.0,5.0,1931.0,1950.0,0,0,4,2,1,0.000000,1,0,2,1,0,0,2,0.000000,0,0.000000,30.854497,30.854497,0,1,0,1,31.968735,27.422618,0.0,42.118879,0.0,0.0,2.0,0.0,2.0,2.0,1,8.0,1,2.0,1,1,1931.0,1,2.0,468.0,1,0,0,9.486833,0.000000,14.317821,0.0,0.0,0.0,0,0,0,0.0,4.0,2008.0,0,1
9,190.0,0,7.071068,86.139422,0,0,0,0,0,2,0,8,3,1,1,3,5.0,6.0,1939.0,1950.0,0,0,1,1,1,0.000000,1,0,2,1,0,0,0,29.171904,0,0.000000,11.832160,31.480152,0,0,0,0,32.817678,0.000000,0.0,32.817678,1.0,0.0,1.0,0.0,2.0,2.0,1,5.0,0,2.0,1,0,1939.0,0,1.0,205.0,2,0,0,0.000000

In [99]:
x_te.head(10)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20.0,0,8.944272,107.805380,0,0,0,0,0,0,0,0,0,0,0,0,5.0,6.0,1961.0,1961.0,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,21.633308,0,12.000000,16.431677,882.0,0,0,0,0,29.933259,0.000000,0.0,29.933259,0.0,0.0,1.0,0.0,2.0,1.0,0,5.0,0,0.0,0,0,1961.0,0,1.0,730.0,0,0,0,11.832160,0.000000,0.0,0.0,10.954451,0.0,0,0,0,0.000000,6.0,2010.0,0,0
1,20.0,1,9.000000,119.444548,0,0,1,0,0,1,0,0,1,0,0,0,6.0,6.0,1958.0,1958.0,1,0,1,1,1,10.392305,0,0,0,0,0,0,1,30.380915,1,0.000000,20.149442,1329.0,0,0,0,0,36.455452,0.000000,0.0,36.455452,0.0,0.0,1.0,1.0,3.0,1.0,1,6.0,0,0.0,0,0,1958.0,0,1.0,312.0,0,0,0,19.824228,6.000000,0.0,0.0,0.000000,0.0,0,1,1,111.803399,6.0,2010.0,0,0
2,60.0,1,8.602325,117.601020,0,0,1,0,0,0,0,1,1,0,0,1,5.0,5.0,1997.0,1998.0,0,0,0,0,0,0.000000,0,0,1,1,0,0,2,28.124722,1,0.000000,11.704700,928.0,0,1,0,0,30.463092,26.476405,0.0,40.360872,0.0,0.0,2.0,1.0,3.0,1.0,0,6.0,0,1.0,1,0,1997.0,1,2.0,482.0,0,0,0,14.560220,5.830952,0.0,0.0,0.000000,0.0,0,0,0,0.000000,3.0,2010.0,0,0
3,60.0,1,8.831761,99.889939,0,0,1,0,0,0,0,1,1,0,0,1,6.0,6.0,1998.0,1998.0,0,0,0,0,1,4.472136,0,0,1,0,0,0,2,24.535688,1,0.000000,18.000000,926.0,0,2,0,0,30.430248,26.038433,0.0,40.049969,0.0,0.0,2.0,1.0,3.0,1.0,1,7.0,0,1.0,2,0,1998.0,1,2.0,470.0,0,0,0,18.973666,6.000000,0.0,0.0,0.000000,0.0,0,1,0,0.000000,6.0,2010.0,0,0
4,120.0,1,6.557439,70.746025,0,0,1,1,0,0,0,2,1,0,1,0,8.0,5.0,1992.0,1992.0,0,0,2,2,0,0.000000,1,0,1,1,0,0,1,16.217275,1,0.000000,31.890437,1280.0,0,2,0,0,35.777088,0.000000,0.0,35.777088,0.0,0.0,2.0,0.0,2.0,1.0,1,5.0,0,0.0,0,0,1992.0,2,2.0,506.0,0,0,0,0.000000,9.055385,0.0,0.0,12.000000,0.0,0,1,0,0.000000,1.0,2010.0,0,0
5,60.0,1,8.660254,100.000000,0,0,1,0,0,1,0,1,1,0,0,1,6.0,5.0,1993.0,1994.0,0,0,2,2,0,0.000000,0,0,1,1,0,0,3,0.000000,1,0.000000,27.622455,763.0,0,1,0,0,27.622455,29.866369,0.0,40.681691,0.0,0.0,2.0,1.0,3.0,1.0,0,7.0,0,1.0,1,0,1993.0,1,2.0,440.0,0,0,0,12.529964,9.165151,0.0,0.0,0.000000,0.0,0,1,0,0.000000,4.0,2010.0,0,0
6,20.0,1,8.185353,89.330846,0,0,1,0,0,0,0,1,1,0,0,0,6.0,7.0,1992.0,2007.0,0,0,2,2,0,0.000000,0,1,1,1,0,0,1,30.577770,1,0.000000,15.264338,1168.0,0,2,0,0,34.452866,0.000000,0.0,34.452866,1.0,0.0,2.0,0.0,3.0,1.0,0,6.0,0,0.0,0,0,1992.0,1,2.0,420.0,0,0,0,21.977261,4.582576,0.0,0.0,0.000000,0.0,0,2,2,22.360680,3.0,2010.0,0,0
7,60.0,1,7.937254,91.662424,0,0,1,0,0,0,0,1,1,0,0,1,6.0,5.0,1998.0,1998.0,0,0,0,0,0,0.000000,0,0,1,1,0,0,3,0.000000,1,0.000000,28.089144,789.0,0,1,0,0,28.089144,26.000000,0.0,38.275318,0.0,0.0,2.0,1.0,3.0,1.0,0,7.0,0,1.0,2,0,1998.0,1,2.0,393.0,0,0,0,0.000000,8.660254,0.0,0.0,0.000000,0.0,0,1,0,0.000000,5.0,2010.0,0,0
8,20.0,1,9.219544,100.876162,0,0,0,0,0,0,0,1,1,0,0,0,7.0,5.0,1990.0,1990.0,0,0,2,2,0,0.000000,0,0,1,1,0,1,2,25.238859,1,0.000000,25.748786,1300.0,0,1,0,0,36.619667,0.000000,0.0,36.619667,1.0,0.0,1.0,1.0,2.0,1.0,1,5.0,0,1.0,3,0,1990.0,0,2.0,506.0,0,0,0,13.856406,0.000000,0.0,0.0,0.000000,0.0,0,1,0,0.000000,2.0,2010.0,0,0
9,20.0,1,8.366600,91.651514,0,0,0,0,0,1,0,0,1,0,0,0,4.0,5.0,1970.0,1970.0,0,0,3,3,0,0.000000,0,0,0,0,0,0,1,28.354894,2,8.831761,0.000000,882.0,0,0,0,0,29.698485,0.000000,0.0,29.698485,1.0,0.0,1.0,0.0,2.0,1.0,0,4.0,0,0.0,0,0,1970.0,1,2.0,525.0,0,0,0,15.491933,0

In [100]:
scaling_features = x_tr.columns
scaling_features_te = x_te.columns
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_tr[scaling_features])
x_test = scaler.fit_transform(x_te[scaling_features])

In [101]:
print(x_test[:5, :10])
x_train[:5, :10]

[[0.         0.         0.45626538 0.34807765 0.         0.
  0.         0.         0.         0.        ]
 [0.         0.25       0.46209494 0.40639977 0.         0.
  0.33333333 0.         0.         0.25      ]
 [0.23529412 0.25       0.42049525 0.39716213 0.         0.
  0.33333333 0.         0.         0.        ]
 [0.23529412 0.25       0.4444959  0.30841457 0.         0.
  0.33333333 0.         0.         0.        ]
 [0.58823529 0.25       0.20658512 0.16237881 0.         0.
  0.33333333 0.33333333 0.         0.        ]]


array([[0.23529412, 0.        , 0.26543756, 0.13056727, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.33271947, 0.14471978, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.25      ],
       [0.23529412, 0.        , 0.27946992, 0.16361818, 0.        ,
        0.        , 0.33333333, 0.        , 0.        , 0.        ],
       [0.29411765, 0.        , 0.2413102 , 0.14412269, 0.        ,
        0.        , 0.33333333, 0.        , 0.        , 0.5       ],
       [0.23529412, 0.        , 0.34956863, 0.19481601, 0.        ,
        0.        , 0.33333333, 0.        , 0.        , 0.25      ]])

In [102]:
x_train_df = pd.DataFrame(x_train, columns = scaling_features)
y_train_df = pd.read_csv('train.csv')['SalePrice']
x_test_df = pd.DataFrame(x_test, columns = scaling_features_te)

In [103]:
# np.random.seed(1)
sel_features = SelectFromModel(Lasso(alpha = 1000, random_state = 0))
sel_features.fit(x_train_df, y_train_df)

SelectFromModel(estimator=Lasso(alpha=1000, random_state=0))

In [104]:
column = x_train_df.columns
x_train_df = x_train_df[column[sel_features.get_support()]]
x_test_df = x_test_df[column[sel_features.get_support()]]

In [105]:
x_train_df.head(10)

,MSSubClass,BldgType,OverallQual,YearBuilt,YearRemodAdd,RoofStyle,MasVnrArea,BsmtQual,BsmtExposure,BsmtFinType1,BsmtFinSF1,HeatingQC,1stFlrSF,GrLivArea,FullBath,KitchenQual,Fireplaces,FireplaceQu,GarageFinish,GarageCars,WoodDeckSF,OpenPorchSF
0,0.235294,0.00,0.666667,0.949275,0.883333,0.0,0.350000,0.00,0.00,0.000000,0.353679,0.00,0.218663,0.406007,0.666667,0.000000,0.000000,0.0,0.000000,0.50,0.000000,0.333942
1,0.000000,0.00,0.555556,0.753623,0.433333,0.0,0.000000,0.00,0.25,0.166667,0.416271,0.00,0.343451,0.303478,0.666667,0.333333,0.333333,0.2,0.000000,0.50,0.589682,0.000000
2,0.235294,0.00,0.666667,0.934783,0.866667,0.0,0.318198,0.00,0.50,0.000000,0.293444,0.00,0.240048,0.421999,0.666667,0.000000,0.333333,0.2,0.000000,0.50,0.000000,0.277096
3,0.294118,0.00,0.666667,0.311594,0.333333,0.0,0.000000,0.25,0.00,0.166667,0.195629,0.25,0.253359,0.407495,0.333333,0.000000,0.333333,0.4,0.333333,0.75,0.000000,0.252953
4,0.235294,0.00,0.777778,0.927536,0.833333,0.0,0.467707,0.00,0.75,0.000000,0.340665,0.00,0.309865,0.503314,0.666667,0.000000,0.333333,0.2,0.000000,0.75,0.473326,0.391874
5,0.176471,0.00,0.444444,0.876812,0.750000,0.0,0.000000,0.00,0.00,0.000000,0.360132,0.00,0.197876,0.327769,0.333333,0.333333,0.000000,0.0,0.333333,0.50,0.216043,0.234189
6,0.000000,0.00,0.777778,0.956522,0.916667,0.0,0.340955,0.50,0.75,0.000000,0.492502,0.00,0.455624,0.402596,0.666667,0.000000,0.333333,0.4,0.000000,0.50,0.545481,0.322808
7,0.235294,0.00,0.666667,0.731884,0.383333,0.0,0.387298,0.00,0.50,0.166667,0.390124,0.00,0.298590,0.482794,0.666667,0.333333,0.666667,0.2,0.000000,0.50,0.523653,0.610691
8,0.176471,0.00,0.666667,0.427536,0.000000,0.0,0.000000,0.25,0.00,0.333333,0.000000,0.25,0.272648,0.419497,0.666667,0.333333,0.666667,0.2,0.333333,0.50,0.324064,0.000000
9,1.000000,0.25,0.444444,0.485507,0.000000,0.0,0.000000,0.25,0.00,0.000000,0.388303,0.00,0.289552,0.255852,0.333333,0.333333,0.666667,0.2,0.000000,0.25,0.000000,0.085514


In [106]:
x_test_df.head(10)

,MSSubClass,BldgType,OverallQual,YearBuilt,YearRemodAdd,RoofStyle,MasVnrArea,BsmtQual,BsmtExposure,BsmtFinType1,BsmtFinSF1,HeatingQC,1stFlrSF,GrLivArea,FullBath,KitchenQual,Fireplaces,FireplaceQu,GarageFinish,GarageCars,WoodDeckSF,OpenPorchSF
0,0.000000,0.00,0.444444,0.625954,0.183333,0.0,0.000000,0.00,0.00,0.000000,0.341626,0.00,0.190587,0.190587,0.25,0.00,0.00,0.0,0.000000,0.2,0.313552,0.000000
1,0.000000,0.00,0.555556,0.603053,0.133333,0.5,0.289346,0.00,0.00,0.166667,0.479765,0.00,0.317961,0.317961,0.25,0.25,0.00,0.0,0.000000,0.2,0.525341,0.220267
2,0.235294,0.00,0.444444,0.900763,0.800000,0.0,0.000000,0.25,0.00,0.333333,0.444136,0.25,0.200934,0.394231,0.50,0.00,0.25,0.2,0.333333,0.4,0.385845,0.214061
3,0.235294,0.00,0.555556,0.908397,0.800000,0.0,0.124515,0.00,0.00,0.333333,0.387459,0.50,0.200293,0.388160,0.50,0.25,0.25,0.4,0.333333,0.4,0.502801,0.220267
4,0.588235,0.25,0.777778,0.862595,0.700000,0.0,0.000000,0.25,0.00,0.166667,0.256098,0.50,0.304713,0.304713,0.50,0.25,0.00,0.0,0.666667,0.4,0.000000,0.332434
5,0.235294,0.00,0.555556,0.870229,0.733333,0.0,0.000000,0.25,0.00,0.500000,0.000000,0.25,0.145459,0.400497,0.50,0.00,0.25,0.2,0.333333,0.4,0.332043,0.336463
6,0.000000,0.00,0.555556,0.862595,0.950000,0.0,0.000000,0.25,0.00,0.166667,0.482874,0.50,0.278852,0.278852,0.50,0.00,0.00,0.0,0.333333,0.4,0.582396,0.168232
7,0.235294,0.00,0.555556,0.908397,0.800000,0.0,0.000000,0.25,0.00,0.500000,0.000000,0.25,0.154573,0.353502,0.50,0.00,0.25,0.4,0.333333,0.4,0.000000,0.317928
8,0.000000,0.00,0.666667,0.847328,0.666667,0.0,0.000000,0.25,0.25,0.333333,0.398564,0.25,0.321168,0.321168,0.25,0.25,0.25,0.6,0.000000,0.4,0.367194,0.000000
9,0.000000,0.00,0.333333,0.694656,0.333333,0.0,0.000000,0.00,0.00,0.166667,0.447771,0.00,0.186002,0.186002,0.25,0.00,0.00,0.0,0.333333,0.4,0.410535,0.000000


In [107]:
y_train_df.head(10)

0    208500
1    181500
2    223500
3    140000
4    250000
5    143000
6    307000
7    200000
8    129900
9    118000
Name: SalePrice, dtype: int64

In [108]:
y = y_train_df.values
y_train = y[:, np.newaxis]
x_train = x_train_df.values
x_test = x_test_df.values

In [109]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(1460, 22)
(1460, 1)
(1459, 22)


variables x_train, y_train are ready to train and x_test is ready to predict

step model development

In [172]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

In [173]:
# np.random.seed(1)
xtr, xte, ytr, yte = train_test_split(x_train, y_train, test_size=0.1, random_state = 0)
print(xtr.shape)
print(yte.shape)

(1314, 22)
(146, 1)


In [174]:
# model = GradientBoostingRegressor(learning_rate = 0.1, verbose = 1, n_estimators=100)
model = RandomForestRegressor()
model.fit(xtr, ytr)
pred = model.predict(xte)
print('Error:', mean_absolute_error(pred, yte))
print('Error %age:', mean_absolute_percentage_error(pred, yte))

C:\Users\EKLAVYA\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Error: 15917.792420091324
Error %age: 0.08421369756220505


In [175]:
# model = GradientBoostingRegressor(learning_rate = 0.1, verbose = 1, n_estimators=100)
model_rf = RandomForestRegressor()
model_rf.fit(x_train, y_train)
pred = model_rf.predict(xte)
print('Error:', mean_absolute_error(pred, yte))
print('Error %age:', mean_absolute_percentage_error(pred, yte))

C:\Users\EKLAVYA\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Error: 6004.93635208741
Error %age: 0.031170488794630573


In [176]:
print(pred[:5])
print(np.squeeze(yte)[:5])

[208222.56 138631.   113769.   200650.8   88706.  ]
[200624 133000 110000 192000  88000]


In [177]:
model_gb = GradientBoostingRegressor()
model_gb.fit(x_train, y_train)
pred = model_gb.predict(xte)
print('Error:', mean_absolute_error(pred, yte))
print('Error %age:', mean_absolute_percentage_error(pred, yte))

C:\Users\EKLAVYA\AppData\Roaming\Python\Python311\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Error: 12203.003921682879
Error %age: 0.07039923516605927


In [178]:
print(pred[:5])
print(np.squeeze(yte)[:5])

[209103.98768513 156674.29628729 117142.07706859 208338.96003552
  90085.46565649]
[200624 133000 110000 192000  88000]


In [179]:
pgb = model_gb.predict(x_test)
prf = model_rf.predict(x_test)
p = np.divide(pgb+prf, 2)
print(p)

[125764.65485053 145049.54482414 181497.91106474 ... 150187.92848318
 119323.27662756 247881.0833786 ]


In [180]:
sub = pd.read_csv('sample_submission_main.csv')
sub.head()

,Id
0,1461
1,1462
2,1463
3,1464
4,1465


In [181]:
sub['SalePrice'] = p
sub.to_csv('sub12.csv', index=False)

sub12.csv is best performing predictions